In [18]:
import random
import torch
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from pytorch_tabnet.tab_model import TabNetRegressor

dataframe = pd.read_csv('data/raw/kbo_batting_stats_by_season_1982-2025.csv')

#### Batting Model (Regression) 

input = batting stas

target = [WAR, WRC+]

- **WAR** : **Wins Above Replacement**, a premier, all-encompassing sabermetric statistic measuring a player’s total value by quantifying how many more wins they provide to their team compared to a readily available "replacement-level" player
- **WRC+** : **Weighted Runs Created Plus**, a stat that measures a hitter's overall offensive value compared to league average. It takes all of a hitter's contributions at the plate and translates that to his impact on runs created for his team

Source
- https://medium.com/@turkishtechnology/deep-learning-with-tabnet-b881236e28c1
- "TabNet: Attentive Interpretable Tabular Learning", Sercan O. Arık, Tomas Pfister, Google Cloud AI

In [24]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetRegressor

# 1. 데이터 로드 및 타겟 설정
# [선택 이유] KBO 역사(1982-2025)를 담은 시계열적 성격이 강한 데이터셋입니다.
try:
    df = pd.read_csv("data/raw/kbo_batting_stats_by_season_1982-2025.csv")
except FileNotFoundError:
    # 파일이 없을 경우를 대비한 가상 구조 (테스트용)
    print("파일을 찾을 수 없습니다. 예시 데이터 구조로 진행합니다.")
    pass

TARGET = "wRC+"
df = df.dropna(subset=[TARGET]).copy()

# 2. 피처 엔지니어링 및 불필요한 컬럼 제거
# [선택 이유] 'Id'는 순수 식별자이며, 'Birthdate'는 'Age'와 정보가 겹치므로 모델의 혼란을 줄이기 위해 제거합니다.
# 'Name'은 선수의 고유 능력을 모델이 기억하게 하기 위해 범주형 변수로 유지합니다.
drop_cols = ["Id", "Name", "Birthdate", "School", "Draft", "oWAR", "dWAR", "WAR"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# 3. 시계열 분할 (Time-based Splitting)
# [선택 이유] 스포츠 데이터는 연도별 에이징 커브나 리그 전체의 타고투저 경향이 변하므로 
# 무작위 분할보다 '연도 기반' 분할이 미래 성능 예측에 훨씬 정확합니다.
train_df = df[df["Year"] <= 2022].copy()
valid_df = df[df["Year"] == 2023].copy()
test_df  = df[df["Year"] >= 2024].copy()

# 4. 범주형 변수 처리 (Label Encoding)
# [선택 이유] TabNet은 내부적으로 Embedding 레이어를 가집니다. 
# One-hot encoding을 하면 차원이 너무 커지지만, Label Encoding 후 cat_dims를 알려주면 
# 모델이 각 범주(팀, 포지션 등)의 의미적 거리를 스스로 학습합니다.
cat_cols = ["Handedness", "Team", "Pos."]
cat_cols = [c for c in cat_cols if c in df.columns]

encoders = {}
for c in cat_cols:
    le = LabelEncoder()
    # [중요 수정] 전체 데이터의 범주를 학습하여 인덱스 범위를 확정합니다.
    all_vals = pd.concat([train_df[c], valid_df[c], test_df[c]], axis=0).astype(str).fillna("NA")
    le.fit(all_vals)
    encoders[c] = le
    
    train_df[c] = le.transform(train_df[c].astype(str).fillna("NA"))
    valid_df[c] = le.transform(valid_df[c].astype(str).fillna("NA"))
    test_df[c]  = le.transform(test_df[c].astype(str).fillna("NA"))

# 5. 수치형 변수 결측치 처리
# [선택 이유] TabNet은 결측치에 강한 편이지만, 명시적으로 중앙값(Median) 처리를 하여 
# 이상치(Outlier)의 영향을 최소화하면서 학습 안정성을 높입니다.
feature_cols = [c for c in df.columns if c != TARGET]
for c in feature_cols:
    if c in cat_cols: continue
    median_val = train_df[c].median()
    train_df[c] = train_df[c].fillna(median_val)
    valid_df[c] = valid_df[c].fillna(median_val)
    test_df[c]  = test_df[c].fillna(median_val)

# 6. TabNet 전용 설정 (cat_idxs, cat_dims)
# [선택 이유] TabNet에게 어떤 컬럼이 '범주'이고, 그 범주에 몇 개의 종류가 있는지 알려줘야 
# 내부 어텐션 메커니즘이 특성 선택을 올바르게 수행합니다.
cat_idxs = [i for i, f in enumerate(feature_cols) if f in cat_cols]

# [버그 수정] cat_dims를 train_df가 아닌 encoders의 클래스 개수로 설정해야 합니다.
# train_df에 없는 카테고리가 valid/test에 있을 경우 IndexError가 발생하는 것을 방지합니다.
cat_dims = [len(encoders[c].classes_) for c in cat_cols]

# numpy 변환
X_train = train_df[feature_cols].values
y_train = train_df[TARGET].values.reshape(-1, 1)
X_valid = valid_df[feature_cols].values
y_valid = valid_df[TARGET].values.reshape(-1, 1)
X_test  = test_df[feature_cols].values
y_test  = test_df[TARGET].values.reshape(-1, 1)

# 7. TabNetRegressor 모델 정의
# [파라미터 선택 이유]
model = TabNetRegressor(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=8,         # [이유] 카테고리별로 8차원 공간에 정보를 압축 (팀, 포지션 의미 학습)
    n_d=32, n_a=32,        # [이유] n_d(결정 단계 폭)와 n_a(어텐션 단계 폭)를 같게 설정하는 것이 논문 권장사항
    n_steps=5,             # [이유] 의사결정 나무의 깊이와 유사한 개념. 5단계 정도로 충분히 복잡한 관계 학습
    gamma=1.5,             # [이유] 특정 피처가 반복 선택되는 것을 제어하여 다양한 지표를 고루 보게 함
    n_independent=2,       # [이유] 각 단계에서 독립적으로 학습할 레이어 수
    n_shared=2,            # [이유] 모든 단계에서 공유할 레이어 수 (효율성 향상)
    lambda_sparse=1e-4,    # [이유] 중요한 피처만 쓰도록 유도(Sparsity). 0에 가까울수록 모든 피처를 다 씀
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="entmax",     # [이유] sparsemax보다 더 명확하게 중요한 피처를 선택하는 경향이 있음
)

# 8. 모델 학습 (Fitting)
# [선택 이유] virtual_batch_size는 Ghost Batch Normalization을 구현하며, 
# 데이터셋이 작을 때 과적합을 방지하고 학습을 안정화하는 핵심 요소입니다.
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=30,           # [이유] 30번 동안 성능 개선 없으면 조기 종료 (Overfitting 방지)
    batch_size=1024,       # [이유] 사용자의 4096보다 조금 줄여 야구 데이터 규모에 맞게 조정
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# 9. 평가 및 결과 출력
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("\n--- [학습 결과 보고서] ---")
print(f"테스트 RMSE (오차): {rmse:.4f}")
print(f"결정계수 R2 (설명력): {r2:.4f}")

# 10. 특성 중요도 (TabNet의 강점)
# [이유] 어떤 타격 지표가 WAR 예측에 가장 기여했는지 확인 가능합니다.
feat_importances = model.feature_importances_
importance_df = pd.DataFrame({'feature': feature_cols, 'importance': feat_importances})
print("\n--- [상위 5개 중요 피처] ---")
print(importance_df.sort_values(by='importance', ascending=False).head(5))

/opt/homebrew/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 10350.25601| val_0_rmse: 1049.23486|  0:00:00s
epoch 1  | loss: 6680.90642| val_0_rmse: 338.10996|  0:00:01s
epoch 2  | loss: 3207.2881| val_0_rmse: 1026.43198|  0:00:01s
epoch 3  | loss: 1659.91085| val_0_rmse: 529.43342|  0:00:02s
epoch 4  | loss: 798.60811| val_0_rmse: 465.30141|  0:00:02s
epoch 5  | loss: 600.19621| val_0_rmse: 418.96251|  0:00:03s
epoch 6  | loss: 638.65442| val_0_rmse: 443.74549|  0:00:03s
epoch 7  | loss: 542.65289| val_0_rmse: 376.04273|  0:00:03s
epoch 8  | loss: 529.46982| val_0_rmse: 63.25964|  0:00:04s
epoch 9  | loss: 534.33131| val_0_rmse: 60.40094|  0:00:04s
epoch 10 | loss: 531.06708| val_0_rmse: 59.6507 |  0:00:05s
epoch 11 | loss: 522.09255| val_0_rmse: 204.19294|  0:00:05s
epoch 12 | loss: 485.60542| val_0_rmse: 339.71713|  0:00:06s
epoch 13 | loss: 548.07045| val_0_rmse: 477.67848|  0:00:06s
epoch 14 | loss: 576.88401| val_0_rmse: 483.73048|  0:00:07s
epoch 15 | loss: 490.79469| val_0_rmse: 511.27829|  0:00:07s
epoch 16 | loss: 444.

/opt/homebrew/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [25]:
# 정상 동작 버전(수정): 문자열 dtype 때문에 median()이 깨지는 문제 해결 포함
# - 핵심 수정 1) USE_NAME=False일 때 Name을 feature에서 제거
# - 핵심 수정 2) 수치형 결측 처리 전에 non-cat 컬럼을 강제로 to_numeric(coerce)
# - 핵심 수정 3) 중앙값은 numeric dtype에서만 계산(안전장치)

import pandas as pd
import numpy as np
import torch
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetRegressor


def seed_everything(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

# 1) 로드
df = pd.read_csv("data/raw/kbo_batting_stats_by_season_1982-2025.csv")

# 2) 기본 정리: 필요한 컬럼 확인 및 숫자 변환
# 문자열 컬럼(범주/텍스트) 목록
TEXT_COLS = {"Name", "Birthdate", "Handedness", "School", "Draft", "Team", "Pos."}

# 숫자형으로 바뀌어야 하는 컬럼들은 가능한 한 전부 변환(실패는 NaN)
for c in df.columns:
    if c in TEXT_COLS:
        continue
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 필수 컬럼
df = df.dropna(subset=["Id", "Year", "wRC+"]).copy()

# shift를 위해 정렬
df = df.sort_values(["Id", "Year"]).reset_index(drop=True)

# 3) 타겟: 다음 시즌 WAR
df["wRC+_next"] = df.groupby("Id")["wRC+"].shift(-1)

df = df.dropna(subset=["wRC+_next"]).copy()
TARGET = "wRC+_next"

# 4) 누수/중복 컬럼 제거(권장)
drop_cols = ["Birthdate", "School", "Draft", "oWAR", "dWAR", "WAR"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# 5) 범주형 컬럼 구성
USE_NAME = False  # True로 하면 Name을 범주로 사용(암기 성향 강해짐)

cat_cols = ["Handedness", "Team", "Pos."]
if USE_NAME and "Name" in df.columns:
    cat_cols = ["Name"] + cat_cols
cat_cols = [c for c in cat_cols if c in df.columns]

# 6) 피처 컬럼 선택
#  - TARGET은 제외
#  - 현재 시즌 WAR는(강한 힌트) 기본 제외
EXCLUDE_FROM_FEATURES = {TARGET, "wRC+"}

# USE_NAME=False면 Name은 피처에서 완전히 제외(중요!)
if (not USE_NAME) and ("Name" in df.columns):
    EXCLUDE_FROM_FEATURES.add("Name")

feature_cols = [c for c in df.columns if c not in EXCLUDE_FROM_FEATURES]

# 7) 시계열 분할(X의 Year 기준)
train_df = df[df["Year"] <= 2022].copy()
valid_df = df[df["Year"] == 2023].copy()
test_df  = df[df["Year"] == 2024].copy()   # 평가용(정답=2025 WAR)
pred_df  = df[df["Year"] == 2025].copy()   # 2026 예측용(정답 없음)

# 8) 누수 없는 범주 인코딩 + UNK 처리
class SafeLabelEncoder:
    def __init__(self, unk_token="__UNK__"):
        self.unk_token = unk_token
        self.le = LabelEncoder()
        self.known = None

    def fit(self, s: pd.Series):
        vals = s.astype(str).fillna("NA").values
        vals = np.concatenate([vals, np.array([self.unk_token])])
        self.le.fit(vals)
        self.known = set(self.le.classes_)
        return self

    def transform(self, s: pd.Series) -> np.ndarray:
        vals = s.astype(str).fillna("NA").values
        vals = np.array([v if v in self.known else self.unk_token for v in vals], dtype=object)
        return self.le.transform(vals)

    @property
    def classes_(self):
        return self.le.classes_

encoders = {}
for c in cat_cols:
    enc = SafeLabelEncoder().fit(train_df[c])  # train만 fit
    encoders[c] = enc

    train_df[c] = enc.transform(train_df[c])
    valid_df[c] = enc.transform(valid_df[c])
    if len(test_df) > 0:
        test_df[c]  = enc.transform(test_df[c])
    if len(pred_df) > 0:
        pred_df[c]  = enc.transform(pred_df[c])

# 9) 수치형 결측 처리: median() 오류 방지
ADD_MISSING_INDICATORS = True

# feature_cols를 복사해 두고, indicator 추가는 별도 리스트에 누적 후 마지막에 확장(루프 중 append 위험 제거)
missing_indicators = []

for c in feature_cols:
    if c in cat_cols:
        continue

    # ★핵심: 수치형으로 강제 변환(문자열이면 NaN으로 바뀜)
    for dset in [train_df, valid_df, test_df, pred_df]:
        if len(dset) == 0:
            continue
        dset[c] = pd.to_numeric(dset[c], errors="coerce")

    if ADD_MISSING_INDICATORS:
        ind_col = f"{c}_isna"
        missing_indicators.append(ind_col)
        for dset in [train_df, valid_df, test_df, pred_df]:
            if len(dset) == 0:
                continue
            dset[ind_col] = dset[c].isna().astype(int)

    # 중앙값 계산은 numeric에서만 수행
    if not pd.api.types.is_numeric_dtype(train_df[c]):
        # 여기까지 왔는데도 numeric이 아니면, 안전하게 스킵
        # (실제로는 위 to_numeric로 대부분 해결됩니다)
        continue

    median_val = train_df[c].median(skipna=True)
    for dset in [train_df, valid_df, test_df, pred_df]:
        if len(dset) == 0:
            continue
        dset[c] = dset[c].fillna(median_val)

# indicator들을 feature_cols에 마지막에 추가
for ind_col in missing_indicators:
    if ind_col not in feature_cols:
        feature_cols.append(ind_col)

# 10) TabNet cat 설정(순서 정렬 필수)
ordered_cat_cols = [f for f in feature_cols if f in cat_cols]
cat_idxs = [feature_cols.index(c) for c in ordered_cat_cols]
cat_dims = [len(encoders[c].classes_) for c in ordered_cat_cols]

# 11) numpy 변환
X_train = train_df[feature_cols].values
X_valid = valid_df[feature_cols].values

y_train = train_df[TARGET].values.reshape(-1, 1)
y_valid = valid_df[TARGET].values.reshape(-1, 1)

X_test = y_test = None
if len(test_df) > 0:
    X_test = test_df[feature_cols].values
    y_test = test_df[TARGET].values.reshape(-1, 1)

X_pred = None
if len(pred_df) > 0:
    X_pred = pred_df[feature_cols].values

# 12) 모델
model = TabNetRegressor(
    cat_idxs=cat_idxs if len(cat_idxs) > 0 else [],
    cat_dims=cat_dims if len(cat_dims) > 0 else [],
    cat_emb_dim=8,
    n_d=32, n_a=32,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="entmax",
)

# 13) 학습
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=30,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
)

# 14) 평가(2024 -> 2025)
if X_test is not None:
    preds_test = model.predict(X_test)
    rmse = float(np.sqrt(mean_squared_error(y_test, preds_test)))
    r2 = float(r2_score(y_test, preds_test))
    print("\n--- [테스트(2024 -> 2025) 성능] ---")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2  : {r2:.4f}")

# 15) 2026 예측(2025 -> 2026)
if X_pred is not None and len(pred_df) > 0:
    preds_2026 = model.predict(X_pred).reshape(-1)

    # 결과 리포팅(원래 문자열 컬럼은 df에서 유지되어 있을 수 있음)
    # pred_df에서 Name을 제외했더라도, 원 df와 merge해서 출력할 수도 있습니다.
    report_cols = [c for c in ["Id", "Year", "Team", "Age"] if c in pred_df.columns]
    out = pred_df[report_cols].copy()
    out["WAR_2026_pred"] = preds_2026

    print("\n--- [2026 WAR 예측 결과: 상위 20명] ---")
    print(out.sort_values("WAR_2026_pred", ascending=False).head(20).to_string(index=False))

# 16) 중요도
feat_importances = model.feature_importances_
importance_df = pd.DataFrame({"feature": feature_cols, "importance": feat_importances})
print("\n--- [상위 10개 중요 피처] ---")
print(importance_df.sort_values("importance", ascending=False).head(10).to_string(index=False))


/opt/homebrew/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 10005.96028| val_0_rmse: 88.94557|  0:00:00s
epoch 1  | loss: 8456.7981| val_0_rmse: 507.16212|  0:00:00s
epoch 2  | loss: 6609.12288| val_0_rmse: 147.84508|  0:00:01s
epoch 3  | loss: 5345.89443| val_0_rmse: 468.14574|  0:00:01s
epoch 4  | loss: 5289.63914| val_0_rmse: 778.71126|  0:00:02s
epoch 5  | loss: 5174.815| val_0_rmse: 152.28337|  0:00:02s
epoch 6  | loss: 5146.38786| val_0_rmse: 233.23023|  0:00:03s
epoch 7  | loss: 5118.54032| val_0_rmse: 214.79689|  0:00:03s
epoch 8  | loss: 5126.70539| val_0_rmse: 148.02118|  0:00:04s
epoch 9  | loss: 5103.8097| val_0_rmse: 143.32856|  0:00:04s
epoch 10 | loss: 5097.17608| val_0_rmse: 174.685 |  0:00:05s
epoch 11 | loss: 5128.27207| val_0_rmse: 111.53207|  0:00:05s
epoch 12 | loss: 5090.88491| val_0_rmse: 100.28361|  0:00:05s
epoch 13 | loss: 5084.01503| val_0_rmse: 83.76698|  0:00:06s
epoch 14 | loss: 5103.58109| val_0_rmse: 67.28873|  0:00:06s
epoch 15 | loss: 5124.93574| val_0_rmse: 70.67712|  0:00:07s
epoch 16 | loss:

/opt/homebrew/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
